In [8]:
from ultralytics import YOLO
model = YOLO("http://localhost:8000/yolo", task="detect")

In [9]:
import cv2
img = cv2.imread('frame.jpg')

imgs = [img]*20
print(len(imgs))

20


In [10]:
model.predict(imgs)


0: 384x640 17 potatos, 8.5ms
1: 384x640 17 potatos, 8.5ms
2: 384x640 17 potatos, 8.5ms
3: 384x640 17 potatos, 8.5ms
4: 384x640 17 potatos, 8.5ms
5: 384x640 17 potatos, 8.5ms
6: 384x640 17 potatos, 8.5ms
7: 384x640 17 potatos, 8.5ms
8: 384x640 17 potatos, 8.5ms
9: 384x640 17 potatos, 8.5ms
10: 384x640 17 potatos, 8.5ms
11: 384x640 17 potatos, 8.5ms
12: 384x640 17 potatos, 8.5ms
13: 384x640 17 potatos, 8.5ms
14: 384x640 17 potatos, 8.5ms
15: 384x640 17 potatos, 8.5ms
16: 384x640 17 potatos, 8.5ms
17: 384x640 17 potatos, 8.5ms
18: 384x640 17 potatos, 8.5ms
19: 384x640 17 potatos, 8.5ms
Speed: 2.4ms preprocess, 8.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'potato'}
 obb: None
 orig_img: array([[[ 95, 133,  57],
         [ 95, 133,  57],
         [ 95, 133,  57],
         ...,
         [121, 150,  95],
         [121, 150,  95],
         [121, 150,  95]],
 
        [[ 95, 133,  57],
         [ 95, 133,  57],
         [ 95, 133,  57],
         ...,
         [121, 150,  95],
         [121, 150,  95],
         [121, 150,  95]],
 
        [[ 95, 133,  57],
         [ 95, 133,  57],
         [ 95, 133,  57],
         ...,
         [121, 150,  95],
         [121, 150,  95],
         [121, 150,  95]],
 
        ...,
 
        [[117, 146,  73],
         [117, 146,  73],
         [117, 146,  73],
         ...,
         [133, 170, 100],
         [133, 170, 102],
         [133, 170, 100]],
 
        [[117, 145,  75],
         [117, 146,  73],
         [117, 146,  73],
         ...,
         [133, 170

In [11]:
from ultralytics import YOLO

# Load a model
model = YOLO("Models/model_seg_fp32/best_yoloseg.pt")  # load an official model

# Retrieve metadata during export. Metadata needs to be added to config.pbtxt. See next section.
metadata = []


def export_cb(exporter):
    metadata.append(exporter.metadata)


model.add_callback("on_export_end", export_cb)

# Export the model
onnx_file = model.export(format="onnx", dynamic=True, batch=500, opset=19)

Ultralytics 8.3.105 🚀 Python-3.10.12 torch-2.8.0.dev20250408+cu128 CPU (Intel Xeon Gold 5315Y 3.20GHz)


YOLOv8n-seg summary (fused): 85 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs

PyTorch: starting from 'Models/model_seg_fp32/best_yoloseg.pt' with input shape (500, 3, 320, 320) BCHW and output shape(s) ((500, 37, 2100), (500, 32, 80, 80)) (12.8 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.50...
ONNX: export success ✅ 23.3s, saved as 'Models/model_seg_fp32/best_yoloseg.onnx' (12.5 MB)

Export complete (38.0s)
Results saved to /home/paperspace/project_conveyer/Models/model_seg_fp32
Predict:         yolo predict task=segment model=Models/model_seg_fp32/best_yoloseg.onnx imgsz=320  
Validate:        yolo val task=segment model=Models/model_seg_fp32/best_yoloseg.onnx imgsz=320 data=C:\Users\Asus Rog Strix\Desktop\Multimodal\Dataset\data.yaml  
Visualize:       https://netron.app


In [12]:
from pathlib import Path

# Define paths
model_name = "model_seg"
triton_repo_path = Path("tmp") / "triton_repo"
triton_model_path = triton_repo_path / model_name

# Create directories
(triton_model_path / "1").mkdir(parents=True, exist_ok=True)

In [13]:
from pathlib import Path

# Move ONNX model to Triton Model path
Path(onnx_file).rename(triton_model_path / "1" / "model.onnx")

# Create config file
(triton_model_path / "config.pbtxt").touch()

data = """
# Add metadata
parameters {
  key: "metadata"
  value {
    string_value: "%s"
  }
}
""" % metadata[0]  # noqa

with open(triton_model_path / "config.pbtxt", "w") as f:
    f.write(data)

In [6]:
import contextlib
import subprocess
import time

from tritonclient.http import InferenceServerClient

# Define image https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver
tag = "nvcr.io/nvidia/tritonserver:25.03-py3"  # 8.57 GB

# Pull the image
subprocess.call(f"sudo docker pull {tag}", shell=True)

# Run the Triton server and capture the container ID
container_id = (
    subprocess.check_output(
        f"sudo docker run -d --rm --gpus 0 -v $(pwd){triton_repo_path}:/models -p 8000:8000 {tag} tritonserver --model-repository=/models",
        shell=True,
    )
    .decode("utf-8")
    .strip()
)

# Wait for the Triton server to start
triton_client = InferenceServerClient(url="localhost:8000", verbose=False, ssl=False)

# Wait until model is ready
for _ in range(10):
    with contextlib.suppress(Exception):
        assert triton_client.is_model_ready(model_name)
        break
    time.sleep(1)

25.03-py3: Pulling from nvidia/tritonserver
Digest: sha256:517c069d37327883715bb548e5db52ebd5bead0faffb9add3f2eb8aa2b81a047
Status: Image is up to date for nvcr.io/nvidia/tritonserver:25.03-py3
nvcr.io/nvidia/tritonserver:25.03-py3


In [20]:
# Wait for the Triton server to start
triton_client = InferenceServerClient(url="localhost:8000", verbose=False, ssl=False)

# Wait until model is ready
for _ in range(10):
    with contextlib.suppress(Exception):
        assert triton_client.is_model_ready('yolo')
        break
    time.sleep(1)

In [21]:
model = YOLO("http://localhost:8000/yolo", task="detect")

In [ ]:
model.predict(img)

WARNING ⚠️ 'source' is missing. Using 'source=/home/paperspace/project_conveyer/.venv/lib/python3.10/site-packages/ultralytics/assets'.


InferenceServerException: [404] Request for unknown model: 'yolo' is not found